# Get requests to Flemish geolocation service

https://docs.python.org/3.1/howto/urllib2.html

In [57]:
import urllib.request
import urllib.parse
import json

### Details about api if of geopunt
* http://loc.geopunt.be/Help/Api/GET-v2-Location_q_latlon_xy_capakey_poi_c

* international, have a look here: http://www.latlong.net/

In [58]:
def latlon_from_address(address):
      
    url = 'http://loc.geopunt.be/v2/Location?'
    
    #strip spaces for more robustness
    values = {'q' : address.strip()}
    
    data = urllib.parse.urlencode(values)

    #get request is url + data
    #for POST request you need data.encode('utf8') first and use binary_data as a second arg instead of a '+'
    req = urllib.request.Request(url + data)
    
    req.add_header('Content-Type', 'text/json')
    response = urllib.request.urlopen(req)
    binary_response = response.read()
    decoded = binary_response.decode('utf8')
    
    response.close()

    jsonobj = json.loads(decoded)
    
    results = jsonobj['LocationResult']
    
    if len(results) == 0:
        return None
    
    location = results[0]['Location']

    latkeyID = 'Lat_WGS84'
    lonkeyID = 'Lon_WGS84'
    
    return (location[latkeyID], location[lonkeyID])
        


In [59]:
print(latlon_from_address('Ruiselede '))

(51.06302363494435, 3.38665410746602)


In [60]:
import pandas as pd

In [61]:
all_data =  pd.read_json('EVA_restodata.json')

In [62]:
all_data.head(n=3)

,city,name,street,tags,zipcode
0,Neerijse,Biesbemd,Kamstraat 33,"[Approved by EVA, EVA voordeel, Veganvriendelijk]",3040
1,Antwerpen,Falafel Shami,Hoogstraat 47,"[Snack, 100% vegetarisch, Approved by EVA, Veg...",2000
2,Antwerpen,Falafel Tof,Hoogstraat 32,"[Eethuis, Snack, 100% vegetarisch, Approved by...",2000


In [63]:
tags = pd.read_json('EVA_tags.json')

In [64]:
tags.head()

,tag
0,Eethuis
1,EVA voordeel
2,100% vegetarisch
3,Snack
4,Cateraar


In [65]:
all_data['full_address'] = all_data['street'] + ' ' + all_data['zipcode'] + ' ' + all_data['city']

In [66]:
all_data['latlon'] = all_data['full_address'].apply(lambda l: latlon_from_address(l))

In [71]:
all_data['found']=all_data['latlon'].apply(lambda p: p is None)
all_data.rename(columns={'found':'missing'}, inplace=True)

In [74]:
df_notfound = all_data[all_data['missing']]

In [129]:
print(df_notfound.shape)
print(all_data.shape)


(91, 8)
(735, 8)


In [96]:
print(latlon_from_address('Leeuwestraat 23 2000 Antwerpen'))
print(latlon_from_address('Leeuwenstraat 23 2000 Antwerpen'))

city
name
street
tags
zipcode
full_address
latlon
missing
None
(51.218385351929605, 4.399357877940981)


In [103]:
df_notfound[0:5]

,city,name,street,tags,zipcode,full_address,latlon,missing
7,Baal,The Vexican,C. Huysmansstraat 126,"[Cateraar, Snack, 100% vegetarisch, Approved b...",3128,C. Huysmansstraat 126 3128 Baal,None,True
16,Antwerpen,'t Koekebakske,Leeuwestraat 23,"[Eethuis, EVA voordeel]",2000,Leeuwestraat 23 2000 Antwerpen,None,True
24,Ieper,Agadir,Rijselsestraat 42,"[Eethuis, Snack]",8900,Rijselsestraat 42 8900 Ieper,None,True
32,Wetteren,Amuz&Vous,V. Van Sandelaan 33,[Cateraar],9230,V. Van Sandelaan 33 9230 Wetteren,None,True
46,Waregem,BAOBAB Catering,"Mirakelstraat 104A, 8790 Waregem",[Cateraar],8790,"Mirakelstraat 104A, 8790 Waregem 8790 Waregem",None,True


In [119]:
# afkorting, spelfout, spelfout, afkorting, dubbele komma
print(latlon_from_address('Camille Huysmansstraat 126 3128 Tremelo'))
print(latlon_from_address('Leeuwenstraat 23 2000 Antwerpen'))
print(latlon_from_address('Rijselstraat 42 8900 Ieper'))
print(latlon_from_address('Victor Van Sandelaan 33 9230 Wetteren'))
print(latlon_from_address('Mirakelstraat 104A 8790 Waregem'))

None
(50.99981171485267, 4.749558921517834)
None
(51.218385351929605, 4.399357877940981)
None
(50.849543772226255, 2.8863381665928447)
None
(51.00217297024971, 3.8875914088732486)
None
(50.870729494598294, 3.397632962878593)


In [120]:
df_notfound[5:10]

,city,name,street,tags,zipcode,full_address,latlon,missing
62,Sint-Niklaas,Biko,Sacramentstraat 3,"[Eethuis, Snack]",9100,Sacramentstraat 3 9100 Sint-Niklaas,None,True
68,Aalst,Black and White,Koostraat 107,[Eethuis],9300,Koostraat 107 9300 Aalst,None,True
81,Oostende,Brasserie Buyl,A. Buylstraat 44B,[Eethuis],8400,A. Buylstraat 44B 8400 Oostende,None,True
91,Gent,Café De Kleine Kunst,Lousbergkaai 99,"[Eethuis, EVA voordeel]",9000,Lousbergkaai 99 9000 Gent,None,True
98,Gent,Café Parti,Maria-Hendrikaplein 65a,"[Eethuis, Veganvriendelijk]",9000,Maria-Hendrikaplein 65a 9000 Gent,None,True


In [128]:
#spelfout, spelfout, afkorting, prefix, prefix
print(latlon_from_address('Sacramentsstraat 3 9100 Sint-Niklaas'))
print(latlon_from_address('Koolstraat 107 9300 Aalst'))
print(latlon_from_address('Adolf Buylstraat 44B 8400 Oostende'))
print(latlon_from_address('Ferdinand Lousbergkaai 99 9000 Gent'))
print(latlon_from_address('Koningin Maria-Hendrikaplein 65a 9000 Gent'))

(51.16461233026816, 4.142836540827675)
(50.939983118445085, 4.02898384827834)
(51.23115235995707, 2.9150036519826585)
(51.04753532194943, 3.7392711886277534)
(51.036379827654144, 3.7161672129454875)


In [130]:
df_notfound[10:15]

,city,name,street,tags,zipcode,full_address,latlon,missing
100,Brussel,Cafeabc,Phillippe de Champagnestraat 23,"[Eethuis, Snack]",1000,Phillippe de Champagnestraat 23 1000 Brussel,None,True
124,Drogenbos,Cook-Om,Rue Marie Collart,"[Eethuis, EVA voordeel, Veganvriendelijk]",1620,Rue Marie Collart 1620 Drogenbos,None,True
128,Elsene,Crêperie au p’tit Breton,Amerikaansestraat 117,[Snack],1050,Amerikaansestraat 117 1050 Elsene,None,True
132,Mechelen,d'Afspraak,Keizeerstraat 23,[Eethuis],2800,Keizeerstraat 23 2800 Mechelen,None,True
139,Wellen,De Bottelarij,Ulbeekstraat 21,[Eethuis],3830,Ulbeekstraat 21 3830 Wellen,None,True


In [139]:
# spelfout, fr/nl, fr/nl, spelfout, postcode
print(latlon_from_address('Philippe de Champagnestraat 23 1000 Brussel'))
print(latlon_from_address('Marie Collartstraat 1620 Drogenbos'))
print(latlon_from_address('Rue Américaine 117 1050 Elsene'))
print(latlon_from_address('Keizerstraat 23 2800 Mechelen'))
print(latlon_from_address('Ulbeekstraat 21 3832 Wellen'))


(50.84314226758593, 4.347341194697616)
(50.78156305645173, 4.31661255727227)
(50.82249491537016, 4.361301054768735)
(51.0290618159853, 4.488280080616929)
(50.84097654956267, 5.308105034230186)


In [140]:
df_notfound[15:20]

,city,name,street,tags,zipcode,full_address,latlon,missing
140,Brugge,De Bottelier,St.Jacobsstraat 63,[Eethuis],8000,St.Jacobsstraat 63 8000 Brugge,None,True
147,Leuven,De Dijlemolens,Zwarte zustersstraat 16/4,[Eethuis],3000,Zwarte zustersstraat 16/4 3000 Leuven,None,True
153,Sint-Pieters-Rode,De Gelaarsde Kat,Gobbelsrode 68a,[Eethuis],3320,Gobbelsrode 68a 3320 Sint-Pieters-Rode,None,True
161,nvriendelijk,De Grote post,Eethuis,[],Vega,Eethuis Vega nvriendelijk,None,True
163,Brugge - Sint Kruis,De Jonkman,Maaltsesteenweg 438,"[Gastronomisch, Veganvriendelijk]",8310,Maaltsesteenweg 438 8310 Brugge - Sint Kruis,None,True


In [189]:
# afkorting+spelfouten, spelfout, stad,  bad record, contractie
print(latlon_from_address('Sint Jakobstraat 63 8000 Brugge'))
print(latlon_from_address('Zwartzustersstraat 16 3000 Leuven'))
print(latlon_from_address('Gobbelsrode 68a 3220 Holsbeek'))
print(latlon_from_address(''))
print(latlon_from_address('Maalse Steenweg 438 8310 Brugge-Sint Kruis'))


(51.20801546511949, 3.2228642331128525)
(50.87305970607464, 4.696587161773074)
(50.915747648372886, 4.807520754675542)
None
(51.20768556028524, 3.2782982449692675)


In [190]:
df_notfound[20:25]

,city,name,street,tags,zipcode,full_address,latlon,missing
167,Rumbeke,De Kleine Prins,Rijstelstraat 5,[Eethuis],8800,Rijstelstraat 5 8800 Rumbeke,None,True
172,"Vorst, Kempen",De Kruimel,Dikstraat 3,[Eethuis],2430,"Dikstraat 3 2430 Vorst, Kempen",None,True
189,Kortrijk,De Promenade,Veemlarkt 30,"[Eethuis, Snack]",8500,Veemlarkt 30 8500 Kortrijk,None,True
192,Brugge,De Republiek,St. Jacobsstraat 36,"[Eethuis, Snack]",8000,St. Jacobsstraat 36 8000 Brugge,None,True
194,Zoutenaaie,De Reygaerd,Reygaerddijkstraat 22,[Eethuis],8630,Reygaerddijkstraat 22 8630 Zoutenaaie,None,True


In [196]:
print(latlon_from_address('Rijselstraat 5 8800 Rumbeke'))
print(latlon_from_address('Dikstraat 3 2430 Laakdal'))
print(latlon_from_address('Veemarkt 30 8500 Kortrijk'))
print(latlon_from_address('Sint Jacobsstraat 36 8000 Brugge'))
print(latlon_from_address('Reygaerdijkstraat 22 8630 Zoutenaaie'))


(50.940853119308244, 3.123873800952088)
(51.08777822444371, 5.0689052810212445)
(50.82624906812174, 3.2734970141452155)
(51.20801546511949, 3.2228642331128525)
(51.06465342677114, 2.7307744620794177)


In [197]:
df_notfound[25:30]

,city,name,street,tags,zipcode,full_address,latlon,missing
197,Stokrooie,De Stroobander,St. Jozefsplein 13,[Eethuis],3511,St. Jozefsplein 13 3511 Stokrooie,None,True
201,Gent,De Walrus,"Coupure Links 497,",[Eethuis],9000,"Coupure Links 497, 9000 Gent",None,True
224,Hasselt,Double Dragons,Hasseltweg 214,[Eethuis],3500,Hasseltweg 214 3500 Hasselt,None,True
232,Oostende,El Mariachi,Kapucijnestraat 44,[Eethuis],8400,Kapucijnestraat 44 8400 Oostende,None,True
239,Sint-Maria-Horebeke,Elim,Dorpstraat 30,[Eethuis],9667,Dorpstraat 30 9667 Sint-Maria-Horebeke,None,True


In [203]:
print(latlon_from_address('Sint Jozefsplein 13 3511 Stokrooie'))
print(latlon_from_address('Coupure Links 497 9000 Gent'))
print(latlon_from_address('Genkersteenweg 214 3500 Hasselt'))
print(latlon_from_address('Kapucijnenstraat 44 8400 Oostende'))
print(latlon_from_address('Dorpsstraat 30 9667 Sint-Maria-Horebeke'))


(50.966124160569926, 5.279991265198184)
(51.055719748430406, 3.7087946627413633)
(50.94450048610327, 5.361322843381503)
(51.233991155864814, 2.9177648388281234)
(50.83845885800066, 3.688273556090574)


In [204]:
df_notfound[30:35]

,city,name,street,tags,zipcode,full_address,latlon,missing
240,Mechelen,Ellis Gourmet burger,Ijzerleen 10,[Eethuis],2800,Ijzerleen 10 2800 Mechelen,None,True
243,Les Bulles - Nederland,Enig Alternatief,Wielakkerstraat 4,[Eethuis],6811,Wielakkerstraat 4 6811 Les Bulles - Nederland,None,True
247,Elsene,Exki,Elsensesteenweg 12,[Snack],1050,Elsensesteenweg 12 1050 Elsene,None,True
254,Gent,Fabula Rasa,Fr. Lousbergkaai 134,[Eethuis],9000,Fr. Lousbergkaai 134 9000 Gent,None,True
259,Gent,Falafel Punt,Rooiegemlaan 389,"[Eethuis, EVA voordeel, Veganvriendelijk]",9000,Rooiegemlaan 389 9000 Gent,None,True
